In [ ]:
#Scraping all financial data available from Oceans14 website creating dataframes with 'pandas' and ploting data with 'matplotlib', written by Capuccino.
#packages
!pip install selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install pandas_datareader
import pandas_datareader.data as web
from pandas.api.types import CategoricalDtype
import os
from collections import defaultdict
#login
email = input('Email: ')
password = input('Senha: ')
#sector and tickers
s = input('Setor (separado por "/"): ')
tickers = input('Tickers (separado por ",": ')
w = tickers.replace(" ",",")
z = list(map(str,w.split(',')))
#getting correct urls for each ticker
for i in z:
    a=i
    print(a)
#login
    ticker = a
    loginp = 'https://www.oceans14.com.br/usuarios/frmLogin.aspx'
    driver = webdriver.Firefox()
    driver.get(loginp)
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, "#ctl00_conteudoPrincipal_txtLogin").send_keys("trocar1234@hotmail.com")
    driver.find_element(By.CSS_SELECTOR, "#ctl00_conteudoPrincipal_txtSenha").send_keys("onlydata")
    time.sleep(2)
    buttom = driver.find_element(By.CSS_SELECTOR, "#ctl00_conteudoPrincipal_cmdEntrar")
    webdriver.ActionChains(driver).click(buttom).perform()
#inicio
    time.sleep(2)
    lupa = driver.find_element(By.CSS_SELECTOR, ".buscaTopoDesktop")
    webdriver.ActionChains(driver).click(lupa).perform()
    driver.find_element(By.CSS_SELECTOR, "#txtTopoPesquisaDesktop").send_keys(ticker)
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, "#txtTopoPesquisaDesktop").send_keys(Keys.RETURN)
    time.sleep(2)
    geturl = driver.current_url
    if "balanco" in geturl:
        if driver.find_elements(By.CSS_SELECTOR, "#bsPainelLucratividadeBanco > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)"):
            res = driver.find_element(By.CSS_SELECTOR, "#bsPainelLucratividadeBanco > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text
            ind = driver.find_element(By.CSS_SELECTOR, "#abaIndicadores01Banco").text
            driver.quit()
#add label for first column and remove symbols from variables
            resz = "Datas\n"+res.replace(".","").replace(" ","\n").replace("%","").replace(",",".")
            indz = "Datas\n"+ind.replace(".","").replace(" ","\n").replace("%","").replace(",",".")
            reszs = resz.splitlines()
            indzs = indz.splitlines()
#define headers for data
#resultados
            reszsr = defaultdict(list)
            key = None
            reset_key = True
            for item in reszs:
                if str(item).isdigit() or item.startswith('-') or item.endswith('%') or item.find(".")>0:
                    reszsr[key].append(item)
                    reset_key = True
                elif reset_key:
                    key = item
                    reset_key = False
                else:
                    key += f' {item}'
#indicadores
            indzsr = defaultdict(list)
            key = None
            reset_key = True
            for item in indzs:
                if str(item).isdigit() or item.startswith('-') or item.endswith('%') or item.find(".")>0:
                    indzsr[key].append(item)
                    reset_key = True
                elif reset_key:
                    key = item
                    reset_key = False
                else:
                    key += f' {item}'
#creating dataframes
            dfres = pd.DataFrame(reszsr).set_index(["Datas"]).replace('---','0')
            dfind = pd.DataFrame(indzsr).set_index(["Datas"]).replace('---','0')  
#dataframes tò float
            dfresf = dfres.astype(float)
            dfindf = dfind.astype(float)
#rename tabela indicadores
            dfindf.columns = ['LPA','P/L','VPA','P/VP','ROE','Div Yield','Div Payout','Beta','Valorização','Negócios/dia','Volume Diário']
#creating final dataframe
            complete = dfresf.merge(dfindf, left_index=True, right_index=True)
#managing plots
            fig1, f1_axes = plt.subplots(ncols=2, nrows=2, figsize=(30,20))
            fig1.suptitle (a, size=50)
            f1_axes[0, 0].plot(complete['Patrimônio'], lw=2, marker='^', markersize=15, label="Patrimônio")
            f1_axes[0, 0].plot(complete['Resultado bruto'], lw=2, marker='.', markersize=10, label="Resultado bruto")
            f1_axes[0, 0].plot(complete['Lucro líquido'], lw=2, marker='.', markersize=10, label="Lucro líquido")
            f1_axes[0, 1].plot(complete['VPA'], lw=2, marker='.', markersize=10, label="VPA")
            f1_axes[1, 0].plot(complete['P/VP'], lw=2, marker='.', markersize=10, label="P/VP")
            f1_axes[1, 0].plot(complete['Beta'], lw=2, marker='.', markersize=10, label="Beta")
            f1_axes[0, 1].plot(complete['ROE'], lw=2, marker='.', markersize=10, label="ROE")
            f1_axes[1, 1].plot(complete['Valorização'], lw=2, marker='.', markersize=10, label="Valorização")
            f1_axes[0, 0].legend()
            f1_axes[1, 0].legend()
            f1_axes[0, 1].legend()
            f1_axes[1, 1].legend()
#save files
            if not os.path.exists("STOCKBR/"+s+"/"+a):
                os.makedirs("STOCKBR/"+s+"/"+a)
                plt.savefig("STOCKBR/"+s+"/"+a+"/"+a+"data.png")
                complete.to_excel(os.path.join("STOCKBR/"+s+"/"+a, a+".xlsx"),sheet_name=a)
                print("SUCCESS")
                driver.quit()
        else:
            print("MISSING TICKER")
            driver.quit()
    else:
        print("INVALID TICKER")
        driver.quit()
print("FINISHED")
driver.quit()